## VERSION 5: trying to implement multi-model multi agent systems. 

In [ ]:
from openai import OpenAI
import json
import random
import time
from datetime import datetime
import statistics

# Setup client
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key="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",
)

class AdaptiveLearningSystem:
    def __init__(self):
        self.traits = [
            "vocabulary_expansion_count",
            "curiosity_level", 
            "confidence_level",
            "output_compression_ratio",
            "emotional_consistency_score"
        ]
        
        # Track learner progression over time
        self.learning_history = []
        self.current_session = 1
        self.learning_goal = ""
        
        # Task difficulty adaptation
        self.task_difficulty = "medium"
        
        # Multi-agent system components
        self.scoring_agent = ScoringAgent(client)
        self.task_generator_agent = TaskGeneratorAgent(client)
        self.feedback_coach_agent = FeedbackCoachAgent(client)
        
    def generate_adaptive_task(self, goal, current_scores=None, session_num=1):
        """Generate tasks that adapt based on learner's current performance"""
        
        # Use Task Generator Agent for more intelligent task creation
        if session_num > 1 and current_scores:
            return self.task_generator_agent.generate_intelligent_task(
                goal, current_scores, self.task_difficulty, session_num
            )
        
        # Fallback to static task pools for first session
        beginner_tasks = [
            "Write 2-3 sentences about why you want to learn this topic.",
            "Describe one thing you already know about this subject.",
            "What questions do you have about this topic?",
            "Write a short paragraph about how you learn best."
        ]
        
        intermediate_tasks = [
            "Recall the last time you felt stuck while learning something similar. What helped you move forward?",
            "Describe a time you had to explain something complex to someone else. What approach did you use?",
            "Write about a moment when you changed your understanding of something. What triggered the shift?",
            "Compare two different ways you could approach learning this topic."
        ]
        
        advanced_tasks = [
            "Analyze a recent challenge you faced and create a systematic approach to solve similar problems.",
            "Write a detailed reflection on how your learning style has evolved over time.",
            "Construct an argument for why someone should learn this topic, addressing potential counterarguments.",
            "Design a learning plan for someone else who wants to master this topic."
        ]
        
        # Adapt task difficulty based on performance
        if current_scores:
            avg_score = statistics.mean(current_scores.values())
            if avg_score < 0.4:
                self.task_difficulty = "beginner"
            elif avg_score > 0.7:
                self.task_difficulty = "advanced"
            else:
                self.task_difficulty = "intermediate"
        
        # Select appropriate task pool
        if self.task_difficulty == "beginner":
            task_pool = beginner_tasks
        elif self.task_difficulty == "advanced":
            task_pool = advanced_tasks
        else:
            task_pool = intermediate_tasks
            
        return random.choice(task_pool)

    def score_response(self, goal, task, response):
        """Agent 1: Scoring Engine - Use LLM-based scoring"""
        return self.scoring_agent.score_response(goal, task, response, self.traits)

    def generate_adaptive_feedback(self, scores, goal, session_num):
        """Agent 3: Feedback Coach - Generate personalized feedback"""
        return self.feedback_coach_agent.generate_feedback(
            scores, goal, session_num, self.task_difficulty, self.learning_history
        )
    
    def track_progress(self, scores, task, response):
        """Track learner progress over time"""
        session_data = {
            "session": self.current_session,
            "timestamp": datetime.now().isoformat(),
            "task": task,
            "response": response,
            "scores": scores,
            "difficulty": self.task_difficulty,
            "avg_score": statistics.mean(scores.values())
        }
        
        self.learning_history.append(session_data)
        
        # Show progress trends
        if len(self.learning_history) > 1:
            prev_avg = self.learning_history[-2]["avg_score"]
            current_avg = session_data["avg_score"]
            trend = "📈 Improving" if current_avg > prev_avg else "📉 Declining" if current_avg < prev_avg else "➡️ Steady"
            print(f"\n📊 PROGRESS TREND: {trend}")
            print(f"Previous avg: {prev_avg:.2f} → Current avg: {current_avg:.2f}")
    
    def run_learning_cycle(self):
        """Main learning loop - the core of the adaptive system"""
        
        print("🎯 ADAPTIVE LEARNING SYSTEM INITIALIZED")
        print("=" * 50)
        
        # Initial setup
        if not self.learning_goal:
            self.learning_goal = input("What are you trying to learn today? ")
        
        print(f"\n🚀 Starting adaptive learning cycle for: {self.learning_goal}")
        print("Type 'quit' to exit at any time.\n")
        
        while True:
            print(f"\n{'='*20} SESSION {self.current_session} {'='*20}")
            
            # Step 1: Generate adaptive task (Agent 2)
            previous_scores = self.learning_history[-1]["scores"] if self.learning_history else None
            task = self.generate_adaptive_task(self.learning_goal, previous_scores, self.current_session)
            
            print(f"📝 TASK (Difficulty: {self.task_difficulty.upper()}):")
            print(f"{task}")
            
            # Step 2: Get user response
            print(f"\n✍️ Your response:")
            user_response = input()
            
            if user_response.lower() == 'quit':
                print("📊 Final learning summary:")
                self.show_progress_summary()
                break
                
            # Step 3: Score response (Agent 1)
            print("\n🔄 Analyzing your response...")
            scores = self.score_response(self.learning_goal, task, user_response)
            
            # Step 4: Generate adaptive feedback (Agent 3)
            feedback = self.generate_adaptive_feedback(scores, self.learning_goal, self.current_session)
            
            # Step 5: Track progress
            self.track_progress(scores, task, user_response)
            
            # Step 6: Display results
            print("\n📊 CURRENT METRICS:")
            for trait, value in scores.items():
                status = "🟢" if value > 0.7 else "🟡" if value > 0.4 else "🔴"
                print(f"{status} {trait}: {value:.2f}")
            
            print(f"\n🤖 ADAPTIVE FEEDBACK:")
            print(feedback)
            
            # Step 7: Prepare for next iteration
            self.current_session += 1
            
            print(f"\n⏱️  Press Enter to continue to session {self.current_session} (or type 'quit' to exit)")
            continue_input = input()
            if continue_input.lower() == 'quit':
                print("📊 Final learning summary:")
                self.show_progress_summary()
                break
    
    def show_progress_summary(self):
        """Display overall learning progress"""
        if not self.learning_history:
            print("No learning data to display.")
            return
            
        print(f"\n🎯 LEARNING SUMMARY for: {self.learning_goal}")
        print("=" * 50)
        
        # Calculate progress trends
        first_session = self.learning_history[0]
        last_session = self.learning_history[-1]
        
        print(f"📈 Sessions completed: {len(self.learning_history)}")
        print(f"📊 Overall improvement: {last_session['avg_score'] - first_session['avg_score']:.2f}")
        
        # Show trait improvements
        print("\n📋 TRAIT IMPROVEMENTS:")
        for trait in self.traits:
            first_score = first_session['scores'][trait]
            last_score = last_session['scores'][trait]
            improvement = last_score - first_score
            trend = "📈" if improvement > 0 else "📉" if improvement < 0 else "➡️"
            print(f"{trend} {trait}: {first_score:.2f} → {last_score:.2f} ({improvement:+.2f})")


# 🧠 AGENT 1: SCORING ENGINE
class ScoringAgent:
    def __init__(self, client):
        self.client = client
        self.models_to_try = [
            "openai/gpt-4o-mini",
            "openai/gpt-3.5-turbo",
            "meta-llama/llama-3.1-8b-instruct:free",
            "google/gemma-2-9b-it:free"
        ]
    
    def score_response(self, goal, task, response, traits):
        """Use LLM to analyze and score user response"""
        
        scoring_prompt = f"""You are an intelligent evaluator trained to assess written learning responses.

A learner is trying to get better at: "{goal}".

They were given this task:
"{task}"

Here is their response:
"{response}"

Evaluate the response on these 5 metrics from 0.0 to 1.0. Return ONLY valid JSON:

1. vocabulary_expansion_count: Variety and sophistication of vocabulary used
2. curiosity_level: Depth of reflection and insight demonstrated  
3. confidence_level: Clarity and assertiveness of expression
4. output_compression_ratio: Conciseness and clarity (higher = more concise)
5. emotional_consistency_score: Appropriate and stable emotional tone

Format: {{"vocabulary_expansion_count": 0.X, "curiosity_level": 0.X, "confidence_level": 0.X, "output_compression_ratio": 0.X, "emotional_consistency_score": 0.X}}"""

        # Try multiple models for reliability
        for model in self.models_to_try:
            try:
                print(f"🔍 Scoring with: {model}")
                
                completion = self.client.chat.completions.create(
                    model=model,
                    messages=[
                        {"role": "system", "content": "You are a precise assessment tool. Return only valid JSON with no additional text."},
                        {"role": "user", "content": scoring_prompt}
                    ],
                    temperature=0.3,
                    max_tokens=200
                )
                
                raw_response = completion.choices[0].message.content.strip()
                
                # Extract JSON from response
                json_start = raw_response.find('{')
                json_end = raw_response.rfind('}') + 1
                
                if json_start != -1 and json_end != -1:
                    json_str = raw_response[json_start:json_end]
                    scores = json.loads(json_str)
                    
                    # Validate all traits present
                    if all(trait in scores for trait in traits):
                        print(f"✅ Successfully scored with {model}")
                        return scores
                
            except Exception as e:
                print(f"❌ {model} failed: {e}")
                continue
        
        # Fallback to intelligent text analysis
        print("⚠️ All models failed, using intelligent fallback")
        return self._fallback_scoring(response, traits)
    
    def _fallback_scoring(self, response_text, traits):
        """Intelligent fallback scoring based on text analysis"""
        scores = {}
        
        # Vocabulary expansion
        words = response_text.lower().split()
        unique_words = set(words)
        complex_words = [w for w in words if len(w) > 6]
        vocab_score = min(1.0, (len(unique_words) / max(1, len(words))) + (len(complex_words) / max(1, len(words))))
        scores["vocabulary_expansion_count"] = round(vocab_score, 2)
        
        # Curiosity level
        depth_indicators = ["because", "however", "realize", "understand", "discovered", "learned", "noticed", "changed", "shift"]
        curiosity_score = min(1.0, sum(1 for indicator in depth_indicators if indicator in response_text.lower()) / 4)
        scores["curiosity_level"] = round(max(0.3, curiosity_score), 2)
        
        # Confidence level
        confident_phrases = ["i learned", "i realized", "i was able", "i discovered", "i understand", "i believe"]
        confidence_score = min(1.0, sum(1 for phrase in confident_phrases if phrase in response_text.lower()) / 3)
        scores["confidence_level"] = round(max(0.4, confidence_score), 2)
        
        # Output compression ratio
        sentence_count = len([s for s in response_text.split('.') if s.strip()])
        word_count = len(response_text.split())
        if word_count > 0:
            compression_score = min(1.0, sentence_count / (word_count / 12))
        else:
            compression_score = 0.1
        scores["output_compression_ratio"] = round(max(0.3, compression_score), 2)
        
        # Emotional consistency
        emotional_score = 0.6
        if any(word in response_text.lower() for word in ["angry", "frustrated", "confused", "upset"]):
            emotional_score = 0.4
        elif any(word in response_text.lower() for word in ["excited", "happy", "confident", "proud"]):
            emotional_score = 0.8
        scores["emotional_consistency_score"] = round(emotional_score, 2)
        
        return scores


# 🤖 AGENT 2: TASK GENERATOR
class TaskGeneratorAgent:
    def __init__(self, client):
        self.client = client
    
    def generate_intelligent_task(self, goal, current_scores, difficulty, session_num):
        """Generate contextually appropriate tasks based on learner performance"""
        
        low_areas = [k for k, v in current_scores.items() if v < 0.5]
        high_areas = [k for k, v in current_scores.items() if v > 0.7]
        
        task_prompt = f"""You are an expert learning task designer. Create a personalized learning task.

CONTEXT:
- Learning goal: {goal}
- Session: {session_num}
- Difficulty level: {difficulty}
- Learner's weak areas: {low_areas}
- Learner's strong areas: {high_areas}

Create a task that:
1. Matches the {difficulty} difficulty level
2. Specifically targets improvement in weak areas: {low_areas}
3. Builds on their strengths: {high_areas}
4. Is engaging and reflective

Return ONLY the task text, no additional formatting or explanation."""

        try:
            completion = self.client.chat.completions.create(
                model="openai/gpt-4o-mini",
                messages=[
                    {"role": "system", "content": "You are a skilled educational task designer."},
                    {"role": "user", "content": task_prompt}
                ],
                temperature=0.7,
                max_tokens=150
            )
            
            return completion.choices[0].message.content.strip()
            
        except Exception as e:
            print(f"⚠️ Task generation failed: {e}")
            # Fallback to basic task
            return f"Reflect on your progress with {goal}. What specific challenge are you facing, and how might you approach it differently?"


# 🎯 AGENT 3: FEEDBACK COACH
class FeedbackCoachAgent:
    def __init__(self, client):
        self.client = client
    
    def generate_feedback(self, scores, goal, session_num, difficulty, learning_history):
        """Generate personalized, actionable feedback"""
        
        low_scores = {k: v for k, v in scores.items() if v < 0.4}
        high_scores = {k: v for k, v in scores.items() if v > 0.7}
        
        # Calculate improvement trends
        trend_info = ""
        if len(learning_history) > 1:
            prev_scores = learning_history[-2]["scores"]
            improvements = []
            declines = []
            
            for trait in scores:
                change = scores[trait] - prev_scores[trait]
                if change > 0.1:
                    improvements.append(trait)
                elif change < -0.1:
                    declines.append(trait)
            
            trend_info = f"Improvements: {improvements}\nDeclines: {declines}"

        feedback_prompt = f"""You are a personalized learning coach. A student learning "{goal}" just completed session {session_num}.

CURRENT PERFORMANCE:
{json.dumps(scores, indent=2)}

CONTEXT:
- Current difficulty: {difficulty}
- Low performance areas: {list(low_scores.keys())}
- High performance areas: {list(high_scores.keys())}
- {trend_info}

Provide concise, actionable feedback (max 150 words) with:
1. **State Analysis**: What these scores reveal about their learning
2. **Targeted Advice**: Specific actions for the 1-2 lowest areas
3. **Next Focus**: What to prioritize in the next session
4. **Encouragement**: Acknowledge progress and strengths

Be supportive but specific."""

        try:
            completion = self.client.chat.completions.create(
                model="openai/gpt-4o-mini",
                messages=[
                    {"role": "system", "content": "You are an encouraging, insightful learning coach."},
                    {"role": "user", "content": feedback_prompt}
                ],
                temperature=0.6,
                max_tokens=250
            )
            
            return completion.choices[0].message.content.strip()
            
        except Exception as e:
            print(f"⚠️ Feedback generation failed: {e}")
            # Fallback feedback
            return f"""You're showing {', '.join(high_scores.keys()) or 'steady progress'} strengths! 

Focus areas for next session: {', '.join(low_scores.keys()) or 'maintaining consistency'}.

💡 Try to be more specific in your responses and connect your ideas more clearly to your learning goals.

Keep building on your progress! 🚀"""


# Run the system
if __name__ == "__main__":
    system = AdaptiveLearningSystem()
    system.run_learning_cycle()